In [92]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
from model import *
from train import *

from test import test
import string
from utils import *
torch.manual_seed(1337)

In [95]:

def load_data(X,Y,block_size):
    features = []
    target = []
    def s_e(text,i,block_size):
        start = i
        end = i + block_size
        start = min(len(text) - block_size,start)
        end = min(len(text),end)
        return text[start:end]
    for f,t in zip(X,Y):
        for i in range(len(f) - block_size + 1):
            features.append(s_e(f,i,block_size))
            target.append(s_e(t,i,block_size))
    features = torch.tensor(features)
    target = torch.tensor(target)
    return features,target

In [96]:
N = 1
n_embeddings = 32
block_size = 10
n_head = 4
device = "cuda"


In [ ]:
src =[1,2]


1
2


[]

In [97]:
def prepare_data(
        src_path,
        dst_path,
        block_size,
        start_token="$",
        end_token = "^",
        add_start_end_tokens=True,
        device="cpu",
        num_lines=None,
    ):
    src = load_txt(src_path,num_lines,encoding=None)
    dst = load_txt(dst_path,num_lines,encoding=None)
    print(src)
    src_vocab = get_vocab(src) if not add_start_end_tokens else get_vocab(src,start=start_token,end=end_token)
    dst_vocab = get_vocab(dst) if not add_start_end_tokens else get_vocab(dst,start=start_token,end=end_token)

    src = [encode(txt,src_vocab) for txt in src]
    dst = [encode(txt,dst_vocab) for txt in dst]

    #src = list(filter(lambda l:len(l)==block_size,src))
    #dst = list(filter(lambda l:len(l)==block_size,dst))



    src,dst = load_data(src,dst,block_size)
    if add_start_end_tokens:
        src = include_tokens(
            src,
            start_token_index=src_vocab.index(start_token),
        )
        dst = include_tokens(
            dst,
            end_token_index=dst_vocab.index(end_token),
        )
    return src.to(device=device),src_vocab,dst.to(device=device),dst_vocab
src,src_vocab,dst,dst_vocab = prepare_data(
    "./training/news-commentary-v9.fr-en.fr",
    "./training/news-commentary-v9.fr-en.en",
    block_size,
    add_start_end_tokens=True,
    num_lines=1000
)
encoder_vocab_size = len(src_vocab)
decoder_vocab_size = len(dst_vocab)

FileNotFoundError: [Errno 2] No such file or directory: './training/news-commentary-v9.fr-en.fr'

In [ ]:
len(src),len(dst)

(151232, 151232)

In [ ]:
model = AttentiveTranslator(
    1,
    encoder_vocab_size,
    decoder_vocab_size,
    n_embeddings,
    n_head,
    src.shape,
    dst.shape,
    device=device
).to(device=device)

In [103]:
len(load_txt("./resources/news-commentary-v9.fr-en.en",num_lines=None))

183784

In [ ]:
fr_content,en_content = "",""
with open("./training/news-commentary-v9.fr-en.fr","r") as f:
  fr_content = f.read()
with open("./training/news-commentary-v9.fr-en.en","r") as f:
  en_content = f.read()

In [ ]:
!tar -xzf training-parallel-nc-v9.tgz

In [ ]:
optim = torch.optim.Adam(
    model.parameters()
)
loss_fn = torch.nn.functional.cross_entropy
data_loader = arrays_to_loader(src,dst,32)
train(model,optim,loss_fn,1,data_loader,device)

tensor(0.0002)


In [ ]:
model.eval()
model(src[0:1].to(device),dst[0:1].to(device)).argmax(-1),dst[0:1]

(tensor([[ 4, 15, 14, 10, 14, 14, 14,  0, 37, 45, 29]], device='cuda:0'),
 tensor([[ 4, 15, 14, 10, 14, 14, 14,  0, 37, 45, 29]], dtype=torch.int32))

In [ ]:
src_vocab.index("^"),dst_vocab.index("^")

(30, 29)

In [ ]:
def test(
    model,
    input_text,
    block_size,
    src_vocab,
    dst_vocab,
    max_tokens=100,
    start_char="^",
    stop_char="$"
):
    model.eval()
    import torch
    out_text = ""

    for i in range(min(max_tokens, len(input_text) - block_size)):
        text = input_text[i:block_size+i]
        text = encode(text, src_vocab)
        out = encode(start_char*len(text), dst_vocab)
        input = torch.tensor([[30,*text]]).to(device)
        output = torch.tensor([[*out,29]]).to(device)
        print(input.shape,output.shape)
        result = model(input, output)
        char = result[:, -1, :].argmax(-1)
        char = decode(char, dst_vocab)
        if char == stop_char:
            print("incountered")
            break
        out_text += char

    return out_text
test(model,"hello there how are you doing",block_size,src_vocab,dst_vocab)

torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])
torch.Size([1, 11]) torch.Size([1, 11])


'^^^^^^^^^^^^^^^^^^^'